# Housing Market Analysis (Seattle)
## Justin Mai

---

## Table of Contents

- [1: Introduction](#1.-Introduction)
    - [1.1: Objective](#11-objective)
    - [1.2 Research Questions](#12-research-questions)
    - [1.3 Steps](#13-steps)
    - [1.4 Data Collection / Description](#14-data-collectiondescription)
- [2: Data Exploration and Manipulation](#2.-Data-Exploration)
    - [2.1: King County Home Sales](#21-king-county-home-sales-data)
    - [2.2: Seattle-Tacoma-Bellevue Unemployment Rates](#22-unemployment-rates)
    - [2.3: Seattle-Tacoma-Bellevue Average Annual Income](#23-average-annual-income)
    - [2.4: Seattle-Tacoma-Bellevue Resident Population](#24-resident-population)
    - [2.5: Seattle-Tacoma-Bellevue Housing Inventory](#25-housing-inventory)
    - [2.6: 10-Year Real Interest Rate](#26-interest-rates)
- [3: Answering Research Questions](#3.-Research-Questions)
    - [3.1.1: Research Question 1 (Analysis)](#311-what-is-the-rate-of-average-annual-income-growth-vs-the-rate-of-growth-for-home-prices)
    - [3.1.2: Research Question 1 (Summary)](#312-research-question-1-summary)
    - [3.2.1: Research Question 2 (Analysis)](#321-how-have-housing-prices-changed-over-time-in-seattle)
    - [3.2.2: Research Question 2 (Summary)](#322-research-question-2-summary)
    - [3.3.1: Research Question 3 (Analysis)](#331-which-factors-have-the-highest-correlation-to-the-pricing-trends-in-the-seattle-area-best-predictor)
    - [3.3.2: Research Question 3 (Summary)](#332-research-question-3-summary)
- [4: Conclusion](#4-conclusion)


# 1. Introduction

## 1.1 Objective

Analyze housing price trends within the King County area and its correlation to external factors such as interest rates and other economic indicators to understand market dynamics and which factors impact the housing market the most. This analysis is made for home buyers and sellers and real estate professionals.

---

## 1.2 Research Questions

1. What is the rate of average annual income growth vs. the rate of growth for home prices?
2. How have housing prices changed over time in Seattle?
3. Which factors have the highest correlation to the pricing trends in the Seattle area (best predictor)?

Factors: Inflation, Job Growth, GDP Growth, Population (demand), Economic Shocks (pandemic), Income, Inventory, Interest Rates

---

## 1.3 Steps

1. Gather datasets (Housing, Inflation, Income, Interest, etc.)
2. Conduct Exploratory analysis (analyze trends at face value, individually)
3. Answer Research Questions
4. Use scikit-learn to find the best measurement for housing price predictions
5. Summarize findings along the way using markdown
6. Create overall summary of the report
7. Create an all inclusive dashboard that gives key details
8. Conclude report

---

## 1.4 Data Collection/Description
King County Home Sales Data:
https://www.kaggle.com/datasets/andykrause/kingcountysales

- 20+ years of King County, Washington all residential home sales data, from 1999 to 2023

Seattle-Tacoma-Bellevue Unemployment Rates (updated monthly):
https://fred.stlouisfed.org/series/SEAT653URN

Seattle-Tacoma-Bellevue Average Annual Income (updated yearly):
https://fred.stlouisfed.org/series/SEAT653PCPI

Seattle-Tacoma-Bellevue Resident Population (updated monthly):
https://fred.stlouisfed.org/series/STWPOP

Seattle-Tacoma-Bellevue Housing Inventory (updated monthly):
https://fred.stlouisfed.org/series/ACTLISCOUMM42660

10-Year Real Interest Rate:
https://fred.stlouisfed.org/series/REAINTRATREARAT10Y

---

# 2. Data Exploration

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo

pyo.init_notebook_mode(connected=True)

## 2.1 King County Home Sales Data

In [2]:
# Average Housing Prices by City (retrieved dataset using SQL):
average_housing_prices = pd.read_csv('sql-average-housing-prices-by-year.csv')
ahp = average_housing_prices.loc[(average_housing_prices['city'] == 'SEATTLE') | 
                                 (average_housing_prices['city'] == 'BELLEVUE') | 
                                 (average_housing_prices['city'] == 'MERCER ISLAND') | 
                                 (average_housing_prices['city'] == 'FEDERAL WAY')]

In [3]:
# Plot of the Average Housing Prices

fig = px.line(ahp, x='year', y='avg_sale_price', color = 'city')
fig.update_layout(
    title = 'Average Home Sale Prices in Major Cities',
    xaxis_title = 'Year',
    yaxis_title = 'Average Sales Prices ($)',
    hovermode = "x unified",
    legend_title_text = 'City'
)
fig.show()

- Within these major cities (Seattle, Bellevue, and King County as a whole), average home prices have risen steadily every year, with clear spikes in home values from 2020 to 2021
- Home values for each city seem to fluctuate at the same rate (strongly correlated patterns).
- Periods of decline between 2007 and 2009 (due to Great Recession because of risky loans)

In [4]:
# Plot average home prices per sq. ft by region
sqft = pd.read_csv('sql-price-per-sqft.csv')
df = sqft.loc[(sqft['city'] == 'SEATTLE') | (sqft['city'] == 'BELLEVUE') | (sqft['city'] == 'MERCER ISLAND') | (sqft['city'] == 'FEDERAL WAY')]

fig = px.line(df, x='year', y='price_per_sqft', color='city')
fig.update_layout(
    title = 'Average Price per Square Feet ($)',
    xaxis_title = 'Year',
    yaxis_title = 'Price Per Sq. Ft. ($)'
)
fig.show()


- Average price per square footage trends persists between all major cities
- Steep inclines in prices between 2020 - 2021
- Steady incline in prices throughout from 2013 - 2022

In [5]:
# Plot of the house appreciation in each city
house_appreciation = pd.read_csv('sql-house-appreciation.csv')
df = house_appreciation.loc[(house_appreciation['city'] == 'SEATTLE') | (house_appreciation['city'] == 'BELLEVUE') | (house_appreciation['city'] == 'MERCER ISLAND') | (house_appreciation['city'] == 'FEDERAL WAY')]

fig = px.line(df, x='year', y='home_appreciation', color='city')
fig.update_layout(
    title = 'Home Appreciation by Year',
    xaxis_title = 'Year',
    yaxis_title = 'Home Appreciation (%)'
)
fig.show()

This graph demonstrates the home appreciation percentages from one year to the next, a percentage above 1 means that the average house value within a specified city rose in price between the years and a percentage below 1 means that the average house value dropped between years.
- Cities East of Seattle (Bellevue, Mercer Island) has the largest fluctuations in home appreciation percentage
- Seattle itself has the most steady home values through the years
- Home fluctuations change at different rates from city to city but move in similar directions

In [6]:
# Finding the cities with the highest fluctuating home prices
house_appreciation_value = pd.read_csv('sql-house-appreciation-value.csv')

df = house_appreciation_value.groupby(['city']).std().reset_index().sort_values('home_appreciation', ascending=False)[['city', 'home_appreciation']]
df.head()

,city,home_appreciation
14,HUNTS POINT,2.195540e+06
40,YARROW POINT,8.354424e+05
2,BEAUX ARTS,5.237135e+05
22,MEDINA,5.184212e+05
8,CLYDE HILL,3.806500e+05


The standard deviation indicates how much the home appreciation values deviate from the mean on average, so the higher the standard deviation, the greater the fluctuation
- The highest fluctuating cities in home appreciation value all lie in the Bellevue area
- All the cities with the highest fluctuations lie along the lake side

---

## 2.2 Unemployment Rates

In [7]:
unemployment = pd.read_csv('seattle-unemployment-rate.csv')
unemployment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  418 non-null    object 
 1   rate              418 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.7+ KB


In [8]:
# Setting the Observation date to a date variable
unemployment['observation_date'] = pd.to_datetime(unemployment['observation_date'])

# Creating a 'year' column based on the date
unemployment['year'] = unemployment['observation_date'].dt.year

# Getting the average unemployement rate by year
unemployment.groupby(['year']).mean().sort_values('rate', ascending=True).head()

,observation_date,rate
year,,
2019,2019-06-16 12:00:00,3.225000
2022,2022-06-16 12:00:00,3.458333
2007,2007-06-16 12:00:00,3.633333
2023,2023-06-16 12:00:00,3.666667
1998,1998-06-16 12:00:00,3.683333


In [9]:
# Plotting the unemployment rate

fig = px.line(unemployment, x='observation_date', y='rate')
fig.update_layout(
    title = 'Seattle-Tacoma-Bellevue: Unemployment Rate by Year',
    xaxis_title = 'Year',
    yaxis_title = 'Unemployment Rate (%)'
)
fig.show()

- Unemployment rates peaked on April 2020 (16.9%) at the height of the COVID-19 Pandemic

---

## 2.3 Average Annual Income

In [10]:
aa_income = pd.read_csv('seattle-average-annual-income.csv')
aa_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   observation_date  55 non-null     object
 1   annual_income     55 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1012.0+ bytes


In [11]:
aa_income['observation_date'] = pd.to_datetime(aa_income['observation_date'])

aa_income['year'] = aa_income['observation_date'].dt.year

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=aa_income['year'],
        y=aa_income['annual_income'],
        mode='lines+markers'
    )
)
fig.update_layout(
    title='Seattle-Tacoma-Bellevue: Average Annual Income ($)',
    xaxis_title = 'Year',
    yaxis_title = 'Average Annual Income($)'
)
fig.show()

- Average Annual Income has grown at a steady rate from 1970 to about 2010
- From 2010 onwards, average annual income grew at a higher rate
- However, home prices have grown at a much higher rate than annual income

---

## 2.4 Resident Population

In [12]:
population = pd.read_csv('seattle-resident-pop.csv')

population['observation_date'] = pd.to_datetime(population['observation_date'])
population['year'] = population['observation_date'].dt.year
population['pop'] = population['pop'] * 1000
population.head()

,observation_date,pop,year
0,2000-01-01,3052379.0,2000
1,2001-01-01,3094380.0,2001
2,2002-01-01,3121895.0,2002
3,2003-01-01,3138938.0,2003
4,2004-01-01,3163703.0,2004


In [13]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=population['year'],
        y=population['pop'],
        mode='lines+markers'
    )
)
fig.update_layout(
    title='Seattle-Tacoma-Bellevue: Total Population by Year',
    xaxis_title='Year',
    yaxis_title='Population'
)
fig.show()

- Within the Seattle-Tacoma-Bellevue region, population growth has seen a steady inclinde from 2000 - 2020
- Total Population saw it's first decrease between 2020 and 2021
- Total Population stagnated from 2020 - 2023

---

## 2.5 Housing Inventory

In [14]:
inventory = pd.read_csv('seattle-housing-inventory.csv')

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=inventory['observation_date'],
        y=len(inventory['observation_date']) * [0],
        name='Baseline'
    )
)
fig.add_trace(
    go.Scatter(
        x=inventory['observation_date'],
        y=inventory['inventory'],
        mode='lines+markers',
        name='Inventory'
    )
)
fig.update_layout(
    title = 'Active Listing Count (Month-Over-Month)',
    xaxis_title = 'Date',
    yaxis_title = 'Inventory Units'
)
fig.show()


- Housing Inventory spiked on May 2022 (at 79.33 units)
- Housing Inventory has displayed the same pattern each year, with increasing trends to begin the year and declines half way through the year (May - June)
- Housing Inventory is negative throughout the holiday season yearly

---

## 2.6 Interest Rates

In [15]:
interest_rate = pd.read_csv('interest-rate.csv')
interest_rate['observation_date'] = pd.to_datetime(interest_rate['observation_date'])
interest_rate['year'] = interest_rate['observation_date'].dt.year

# Average Interest Rate by Year (Highest)
interest_rate.groupby(['year']).mean().sort_values('rate', ascending=False).head()

,observation_date,rate
year,,
1982,1982-06-16 12:00:00,6.934689
1984,1984-06-16 08:00:00,6.637399
1983,1983-06-16 12:00:00,5.888257
1985,1985-06-16 12:00:00,5.660393
1988,1988-06-16 08:00:00,4.452901


In [16]:
# Average Interest Rate by Year (Lowest)
interest_rate.groupby(['year']).mean().sort_values('rate', ascending=True).head()

,observation_date,rate
year,,
2020,2020-06-16 08:00:00,-0.119478
2021,2021-06-16 12:00:00,-0.031760
2012,2012-06-16 08:00:00,-0.013249
2013,2013-06-16 12:00:00,0.287844
2016,2016-06-16 08:00:00,0.420649


In [17]:
# Plotting 10-Year Real Interest Rate

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=interest_rate['observation_date'],
        y=interest_rate['rate'],
        mode='lines',
        name = 'Interest Rates'
    )
)
fig.update_layout(
    title = '10-Year Real Interest Rate',
    xaxis_title= 'Date',
    yaxis_title= 'Interest Rate (%)',
    xaxis=dict(rangeslider=dict(visible=True))
)

fig.show()

- We saw the highest interest rates in the 1980s
- We saw the lowest interest rates in the 2010s
- Interest rates were on a steady decline up until 2012 where we've seen more large fluctuations and stagnation with recent increasing trends (climbing above 1%)

---

# 3. Research Questions

---

In [18]:
from sklearn.linear_model import LinearRegression

## 3.1.1 What is the rate of average annual income growth vs. the rate of growth for home prices?

In [19]:
df_2000 = aa_income.loc[(aa_income['year'] >= 2000)]
model = LinearRegression().fit(df_2000[['year']], df_2000['annual_income'])

# Slope of the rate of change in annual income
model.coef_[0]


2423.73

In [20]:
df_2010 = aa_income.loc[(aa_income['year'] >= 2010)]
model = LinearRegression().fit(df_2010[['year']], df_2010['annual_income'])

# Slope of the rate of change in annual income
round(model.coef_[0],2)

3800.02

**Rate of Average Annual Growth (2000 - 2023) = 4.31%**

**Rate of Average Annual Growth (2010 - 2023) = 5.7%**

In [21]:
linear_rates = {}
for city, group in average_housing_prices.groupby('city'):
    X = group[['year']]
    y = group['avg_sale_price']

    model = LinearRegression()
    model.fit(X,y)

    linear_rates[city] = model.coef_[0]

linear_rates_df = pd.DataFrame(list(linear_rates.items()), columns=['city', 'rates'])
linear_rates_df['rates'] = round(linear_rates_df['rates'],2)
linear_rates_df.sort_values('rates').head()

,city,rates
35,SKYKOMISH,10721.42
0,ALGONA,11305.75
28,PACIFIC,13064.76
24,MILTON,13665.03
13,FEDERAL WAY,14928.96


In [22]:
linear_rates_df.sort_values('rates', ascending=False).head()

,city,rates
14,HUNTS POINT,255840.16
40,YARROW POINT,195270.24
22,MEDINA,185579.17
8,CLYDE HILL,161834.59
2,BEAUX ARTS,95640.59


In [24]:
# Plotting the two extreme points
fig = px.line(average_housing_prices.loc[(average_housing_prices['city'] == 'HUNTS POINT') | 
                                         (average_housing_prices['city'] == 'SKYKOMISH') |
                                         (average_housing_prices['city'] == 'SEATTLE')]
              , x='year', y='avg_sale_price', color='city')
fig.update_layout(
    title = 'Average Sale Price Trends by Year',
    xaxis_title = 'Year',
    yaxis_title = 'Average Sale Price ($)',
    hovermode= 'x unified'
)
fig.show()

In [25]:
# Average Housing rates starting from 2000 to 2023 (SQL)
average_housing_price_rates_2000 = pd.read_csv('sql-average-housing-price-rates.csv')
average_housing_price_rates_2000 = average_housing_price_rates_2000.loc[average_housing_price_rates_2000['city'] != 'SeaTac']

# Average Housing rates starting from 2010 to 2023
average_housing_price_rates_2010 = pd.read_csv('sql-average-housing-price-rates-2010.csv')
average_housing_price_rates_2010 = average_housing_price_rates_2010.loc[average_housing_price_rates_2010['city'] != 'SeaTac']

In [26]:
# Comparison graph for housing rates starting from 2000 vs. starting from 2010
fig = go.Figure()
fig.add_trace(
    go.Bar(x = average_housing_price_rates_2000['city'],
           y = average_housing_price_rates_2000['avg_rate'],
           name = '2000 to 2023')
)
fig.add_trace(
    go.Bar(x = average_housing_price_rates_2010['city'],
           y = average_housing_price_rates_2010['avg_rate'],
           name = '2010 to 2023')
)
fig.update_layout(
    title = 'Average Rates Comparison by City (2000 vs. 2010)',
    xaxis_title = 'City',
    yaxis_title = 'Rate (%)'
)
fig.show()

## 3.1.2 Research Question 1 Summary

Summary Statistics (Linear Regression):
- Average Annual Income Growth in the King County Area from *2000 - 2023* is **$2,423.73** or an average of **4.31%** per year
- Average Annual Income Growth in the King County Area from *2010 - 2023* is **$3,800.02** or an average of **5.70%** per year
- All cities in King County, regardless of the date of analysis, had an average housing rate **increase of over 5 %**
- Seattle's housing market saw an average of **$32,731.85 increase per year**
- The city of Hunt's Point saw the greatest increase in housing prices per year at an **average of $255,840.16**
- The city of Skyhomish saw the smallest increase in housing prices per year at an **average of $10,721.42**
- Housing prices have grown at a much higher rate from 2010 compared to 2000 for all cities (except for Hunt's Point)
- **Average Annual Income Growth rates have been slower than Average Home Pricing Rates in all cities within the King County**

---

## 3.2.1 How have housing prices changed over time in Seattle?

In [28]:

fig = px.line(average_housing_prices.loc[(average_housing_prices['city'] == 'SEATTLE')], x='year', y='avg_sale_price', color = 'city')
fig.update_layout(
    title = 'Average Home Sale Prices in Major Cities',
    xaxis_title = 'Year',
    yaxis_title = 'Average Sales Prices ($)',
    hovermode = "x unified",
    legend_title_text = 'City'
)
fig.show()

In [27]:
# Average home sales trend in Seattle

linear_rates_df.loc[linear_rates_df['city'] == 'SEATTLE']

,city,rates
33,SEATTLE,32731.85


In [29]:
fig = px.line(sqft.loc[(sqft['city'] == 'SEATTLE')], x='year', y='price_per_sqft', color='city')
fig.update_layout(
    title = 'Average Price per Square Feet ($)',
    xaxis_title = 'Year',
    yaxis_title = 'Price Per Sq. Ft. ($)'
)
fig.show()

## 3.2.2 Research Question 2 Summary

Summary Statistics:
- Home sales in Seattle have **increased by an average of $32,731.85** per year since 2000
- Home sales have **increased by an average rate of 5.92%** per year since 2000
- Home sales saw a decrease between 2007 and 2009 due to The Great Recession
- Seattle reached a high average of **$689.87 per sqft** in 2022

---

## 3.3.1 Which factors have the highest correlation to the pricing trends in the Seattle area (best predictor)?

In [76]:
# Combining dataframes of the possible factors

df1 = unemployment.loc[unemployment['year'] >= 2000]
df1 = df1.groupby(['year']).mean('rate')

df2 = aa_income.loc[aa_income['year'] >= 2000][['year', 'annual_income']].set_index('year')

df3 = population.loc[population['year'] >= 2000][['year', 'pop']].set_index('year')

df4 = interest_rate.loc[interest_rate['year'] >= 2000][['year', 'rate']].groupby(['year']).mean('rate')

df1.rename(columns={'rate': 'unemployment_rate'}, inplace=True)
df4.rename(columns={'rate': 'interest_rate'}, inplace=True)

In [91]:
combined_df = df1.join([df2, df3, df4, average_housing_prices.loc[(average_housing_prices['city'] == 'SEATTLE')][['year','avg_sale_price']].set_index('year')], how='inner')
combined_df.tail()

,unemployment_rate,annual_income,pop,interest_rate,avg_sale_price
year,,,,,
2019,3.225000,76794,3977785.0,0.534974,875070.22
2020,8.425000,81697,4027804.0,-0.119478,926792.36
2021,4.733333,89358,4016479.0,-0.031760,1032881.29
2022,3.458333,92291,4032242.0,1.059399,1133248.27
2023,3.666667,99339,4044837.0,1.657494,1086129.11


In [92]:
combined_df.corr()

,unemployment_rate,annual_income,pop,interest_rate,avg_sale_price
unemployment_rate,1.000000,-0.320591,-0.216886,-0.312289,-0.360693
annual_income,-0.320591,1.000000,0.942012,-0.510874,0.986156
pop,-0.216886,0.942012,1.000000,-0.700264,0.935600
interest_rate,-0.312289,-0.510874,-0.700264,1.000000,-0.477075
avg_sale_price,-0.360693,0.986156,0.935600,-0.477075,1.000000


In [93]:
correlation_matrix = combined_df.corr()

fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.index,
    zmin=-1, zmax=1,
    colorbar=dict(title="Correlation"),
    text=correlation_matrix.values,
    texttemplate="%{text:.2f}"
))

fig.update_layout(
    title="Correlation Matrix Heatmap",
    xaxis_title="Variables",
    yaxis_title="Variables",
    autosize=False,
    width=600,
    height=600
)

fig.show()

## 3.3.2 Research Question 3 Summary
- The average home sales prices are most correlated to annual income trends at **0.99 correlation**
- The average home sales prices are least correlated to unemployment rate at **-0.36 correlation**
- Population is also strongly correlated with average sales prices in Seattle at **0.94 correlation**
- **Annual income in the Seattle area is most indicative (best predictive variable) for sales prices in Seattle**

---

## 4. Conclusion

**Home Appreciation Trends and Fluctuations**

Analysis of home appreciation percentages reveals varied trends across different cities. Cities east of Seattle, like Bellevue and Mercer Island, exhibited the most significant fluctuations in home value appreciation, whereas Seattle displayed more stable home values over the years. The highest standard deviations in home appreciation values were observed in lake-side cities, indicating more considerable volatility in these areas.

Insight: The geographic variation in home value fluctuations suggests that local factors, such as proximity to lakes and urban centers, may drive market dynamics. Understanding these trends can guide targeted housing policies and investment strategies.

**Research Question 1: Trends in Income and Housing Price Growth**

The data reveals significant trends in both income growth and housing price increases in King County from 2000 to 2023. The average annual income growth over this period was 2,423.73 dollars, with a more accelerated growth of 3,800.02 dollars annually from 2010 to 2023. This corresponds to an average yearly increase of 4.31% and 5.70%, respectively. Despite these income gains, housing prices in all cities consistently grew at rates exceeding 5%, with Seattle experiencing an annual increase of $32,731.85.

Hunt’s Point stood out with the highest housing price surge, averaging 255,840.16 dollars per year, whereas Skykomish saw the smallest increase at 10,721.42 dollars annually. Notably, the pace of housing price growth intensified post-2010 across the region, contrasting the slower income growth rates, underscoring a potential affordability challenge for residents.

Insight: The disparity between income growth and housing price escalation highlights the need for strategies to address housing affordability and ensure that income growth can better support homeownership in the area.

**Research Question 2: Home Sales Trends in Seattle**

Seattle's home sales have witnessed a consistent average increase of 32,731.85 dollars per year since 2000, equating to a 5.92% annual growth rate. This trajectory faced a setback during The Great Recession (2007-2009), but recovery and subsequent growth pushed the average price per square foot to 689.87 dollars by 2022.

Insight: The resilience of Seattle’s housing market post-recession indicates strong demand and potential market stability. However, monitoring economic fluctuations is essential to predict and mitigate future downturns.

**Research Question 3: Correlation between Home Sales Prices and Socioeconomic Factors**

The correlation analysis identifies annual income as the most predictive factor for home sales prices in Seattle, with a correlation coefficient of 0.99. Conversely, unemployment rates show the weakest correlation at -0.36. Population growth also strongly correlates with sales prices, at 0.94.

Insight: The strong correlation between income and home prices suggests that economic policies aimed at income growth can directly influence housing affordability. Policymakers should consider this linkage when designing housing and economic development initiatives.